# INRIA ASSIGNEMENT

Le but de ce devoir est dans un premier temps la détection et la suppression d'entrées doublons dans une base de données puis dans un second temps l'exploration et la visualiation des données fournis

In [34]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///data.db', echo=False)
con = engine.connect()
df_patient = pd.read_sql('select * from patient', con=con)
df_pcr = pd.read_sql('select * from test', con=con)
con.close()

Deux bases de données sont fournis : ***df_patient*** et ***df_pcr***.


In [35]:
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
patient_id       20000 non-null int64
given_name       19560 non-null object
surname          19575 non-null object
street_number    19618 non-null float64
address_1        19204 non-null object
suburb           19788 non-null object
postcode         19801 non-null object
state            18010 non-null object
date_of_birth    17989 non-null float64
age              16003 non-null float64
phone_number     19081 non-null object
address_2        7893 non-null object
dtypes: float64(3), int64(1), object(8)
memory usage: 1.8+ MB


In [36]:
df_patient.head(5)

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
0,221958,matisse,clarke,13.0,rene street,ellenbrook,2527,wa,19710708.0,32.0,08 86018809,westella
1,771155,joshua,elrick,23.0,andrea place,east preston,2074,nsw,19120921.0,34.0,02 97793152,foxdown
2,231932,alice,conboy,35.0,mountain circuit,prospect,2305,nsw,19810905.0,22.0,02 20403934,None
3,465838,sienna,craswell,39.0,cumberlegeicrescent,henty,3620,wa,19840809.0,30.0,02 62832318,jodane
4,359178,joshua,bastiaans,144.0,lowrie street,campbell town,4051,nsw,19340430.0,31.0,03 69359594,None


In [37]:
df_pcr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 2 columns):
patient_id    8800 non-null int64
pcr           8800 non-null object
dtypes: int64(1), object(1)
memory usage: 137.6+ KB


In [38]:
df_pcr.head(5)

,patient_id,pcr
0,653091,N
1,347667,Negative
2,708481,Positive
3,148825,Negative
4,150081,Negative


La première base de données est un dataframe contenant des entrées de patients (id,nom,prénoms,adresse,code postal). Certaines colonnes contiennent des entrées nulles. 
La seconde contient des id de patients avec les résultats au test PCR. Elle ne contient pas d'entrées nulles

On va donc commencer par "nettoyer" les deux dataframes.


## 1. Nettoyage du dataframe : ***df_patient***

In [39]:
df_patient.dtypes


patient_id         int64
given_name        object
surname           object
street_number    float64
address_1         object
suburb            object
postcode          object
state             object
date_of_birth    float64
age              float64
phone_number      object
address_2         object
dtype: object

<a id='incoherence_donnees'></a>
On regarde tout d'abord la qualité/fiabilité des données fournis. Cela passe notamment par les types des différentes colonnes d'entrées. 
- ***Patient ID*** : Le numéro ID d'un patient, normalement doit être unique, est en général de type entier. Ce qui est le cas ici
- ***given_name*** : Le prénom officiel du patient est une string donc categorical donc a bien comme *dtype* un objet. De même pour surname; adress_1,adress_2,suburb,state.

- ***phone_number*** : Le numéro de téléphone, peut contenir une extension correspondant à un état ou un espace donc il est préférable de le laisser en string

- ***date_of_birth*** : Les dates de naissances des patients pourraient être sous type Timestamp ou au mieux sous forme de int64, de même pour l'âge et le numéro de rue. 

- ***postcode***  : Le code postal est un nombre allant de 3 à 4 chiffres devrait être sous type int64 et non une categorical value. De plus les states fournis ne correspondent pas aux codes postaux




In [40]:
df_patient.describe()

,patient_id,street_number,date_of_birth,age
count,20000.000000,19618.000000,1.798900e+04,16003.000000
mean,553918.938750,98.812468,1.949638e+07,27.534337
std,260234.961111,2481.908686,2.926916e+05,7.807008
min,100005.000000,0.000000,1.290041e+07,0.000000
25%,330194.250000,10.000000,1.924113e+07,24.000000
50%,556327.500000,24.000000,1.950032e+07,29.000000
75%,779419.500000,60.000000,1.975052e+07,33.000000
max,999993.000000,342951.000000,1.999123e+07,92.000000


D'un point de vue statistique, on constate des valeurs NULL pour les numéros de *rues, dob et l'âge*. Pour la date de naissance on remarque une date de naissance minimale à 1290. Enfin si on compare la moyenne d'âge avec l'âge médian on trouve une différence de 2ans. 

In [41]:
df_patient["age"].median()


29.0

In [42]:
len(df_patient["patient_id"].unique()) #Find how many uniques values in patient_id column

19798

On a 19798 ID de patients distincts au lieu de 20000, or l'ID de patient pour un hôpital est supposée être unique. Cela peut être dû soit à des doublons, soit les données regroupées dans cette base proviennent de différents hôpitaux qui ont chacun leur système numérique unique.

In [44]:
patient_id_dupes = list(df_patient["patient_id"].duplicated()) #Find the duplicates values in the patient_id column
patient_id_dupes_idx = [i for i in range(len(patient_id_dupes)) if patient_id_dupes[i]==True] #Retrieve their index

In [45]:
df_patient.loc[df_patient["patient_id"] == df_patient.loc[patient_id_dupes_idx[0],"patient_id"]]

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
1456,119816,jed,rollins,63.0,mauldon street,macquarie fields,4169,nsw,19010805.0,NaN,07 73786737,None
2377,119816,sam,iasenzaniro,12.0,booroondara street,brighton,6153,sa,19621207.0,NaN,03 90915909,None


Ces deux patients ont le même ID mais sont deux personnes différentes. On peut en conclure que l'ID de patient n'est pas un critère pour identifier un doublons dans notre base de données.

In [46]:
df_patient.describe(include='object')

,given_name,surname,address_1,suburb,postcode,state,phone_number,address_2
count,19560,19575,19204,19788,19801,18010,19081,7893
unique,1171,4273,4326,2925,2021,102,17916,3460
top,emiily,white,paul coe crescent,toowoomba,4350,nsw,02 97010866,brentwood vlge
freq,373,539,57,167,96,6143,4,85


Pour les données non numériques, on remarque pour les états qu'on a 102 valeurs, or seule 10 états existent en Australie.

On va donc procéder la manière suivante pour le nettoyage de données :
<br>
<br>
    A) Remplissage des valeurs NULL pour les colonnes ***given_name, surname, adress_1, address_2, suburb, phone_number***. 
    Les données de ces colonnes non fournis ne peuvent être corrigés autrement.
    <br>
    <br>
    B) Correction des dates de naissances et âges
    <br>
    <br>
    C) Correction des codes postaux et des états

## A) Remplissage

In [47]:
#We work on a instance copy of the dataframe, thus any modifications we can always keep the original one without
#reloading it again
df_dedup = df_patient.copy()

In [48]:
df_dedup["given_name"]=df_dedup['given_name'].fillna('None')
df_dedup["surname"]=df_dedup['surname'].fillna('None')
df_dedup["address_1"]=df_dedup['address_1'].fillna('None')
df_dedup["address_2"]=df_dedup['address_2'].fillna('None')
df_dedup["phone_number"]=df_dedup['phone_number'].fillna(0)
df_dedup["street_number"]=df_dedup['street_number'].fillna(0)

In [49]:
df_dedup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
patient_id       20000 non-null int64
given_name       20000 non-null object
surname          20000 non-null object
street_number    20000 non-null float64
address_1        20000 non-null object
suburb           19788 non-null object
postcode         19801 non-null object
state            18010 non-null object
date_of_birth    17989 non-null float64
age              16003 non-null float64
phone_number     20000 non-null object
address_2        20000 non-null object
dtypes: float64(3), int64(1), object(8)
memory usage: 1.8+ MB


## B) Correction de colonnes
### Etats et codes postaux

Comme on l'a souligné auparavant des [incohérences](#incoherence_donnees) apparraissent dans la correspondance des codes postaux et des états. L'Australie est composé de 8 états ayant chacun un intervalle de données et une abbréviation spécifique. 
***


- __Exemple:__
<br>
<br>
***Premier cas :**
On prend une entrée dans la liste au hasard. Le postcode écrit est *2428* ce qui correspond au state **NSW** or le state écrit est **QLD**. 
***Second cas :**
On prend une entrée dans la liste au hasard. Le state écrit est *vc* ce qui correspond à aucun state  officiellement mais se rapproche du state  **VIC**. 

In [50]:
print(" First Example : ")
print(df_dedup.iloc[15,6:8])
print("  ")
print(" Second Example : ")
print("  ")
print(df_dedup.iloc[301,6:8])


 First Example : 
postcode    2428
state        qld
Name: 15, dtype: object
  
 Second Example : 
  
postcode    4574
state         vc
Name: 301, dtype: object


Pour mettre à jours nos deux colonnes on effectue le procédé suivant : 

1. On commence par vérifier les valeurs NaN dans notre colonne **Postcode** ainsi que les entrées ne correspondant pas à un postcode.On fait pareillement pour les states. On leur assigne une valeur nulle. 

2. On définis un dictionnaire des states avec l'intervalle dans lequel le postcode correspondant se trouve. Pour les postcodes non nulles on assigne les bon states à partir de notre dictionnaire.

3. Pour les postocdes nulles on vérifie si un state est fournis. Dans le cas où le state est fournis on assigne le postcode correspondant. Dans le cas où le state n'est pas fournis, on peut vérifier à partir du suburb : https://jumpshare.com/v/V8Cu6szjbgAzdcMXWfw9# (des BDD sont disponibles sur internet). 

4. Dans le cas où ni le state, ni le postcode ni le surburb n'est fournis, il est préférable de drop l'entrée car le but étant d'étudier la prévalence du COVID en fonction des zones géographiques cet entrée ne sera pas utilisable. *(on constatera par la suite que nous n'arriverons pas à cet étape là, tout les suburb, postcodes et states sont disponibles dans notre dataframe)*

In [51]:
#Create two new column to update our postcode and state and keep the original to compare
df_dedup["postcode_updated"]=df_dedup["postcode"]
df_dedup["state_updated"]=df_dedup["state"]
df_dedup["suburb_updated"]=df_dedup["suburb"]

In [52]:
#List of index of NULL postcode and state
index_null_postcode = df_dedup[df_dedup['postcode'].isnull()].index.tolist()
index_null_state=df_patient[df_patient['state'].isnull()].index.tolist()

In [53]:
#We create a new column to find index of postcode who are different than digits 
df_dedup["postcode_str"]=df_dedup["postcode"].astype(str)
odd_postcode=[i for i in range(len(df_dedup["postcode_str"])) if len(df_dedup.loc[i,"postcode_str"])>4]

In [54]:
print(df_dedup.iloc[odd_postcode[0],5:8])


suburb                      3023
postcode    port noarlunga south
state                        tas
Name: 3976, dtype: object


Le postcode fournis est en réalité une banlieue en Australie situé en Australie du Sud soit un code postale entre 5000 et 6000 or le code postal fournis ici est 3023 correspondant à l'état Victoria. 
On fait donc le choix de ne garder pour ces cas là que le state fournis
<a id='suburb'></a>

In [55]:
#Because all the entry in are swapped we swapped them:
for i in odd_postcode:
    df_dedup.loc[i,"suburb_updated"]=df_dedup.loc[i,"postcode"]
    df_dedup.loc[i,"postcode_updated"]=df_dedup.loc[i,"suburb"]

In [56]:
#Verify that all the suburbs are updated 
for i in odd_postcode:
    print(df_dedup.loc[i,"suburb_updated"])
    

port noarlunga south
naremburn
toowoobma
brighton
oraneg
blacktown
forest hill
port lincoln
ashfield
forest hill
toowoomba
regents park
kelmscott
donnybrook


In [57]:
#We check every postcode assigned is 3-4 digits
odd_postcode_verif=[i for i in odd_postcode if len(df_dedup.loc[i,"postcode_updated"])>4]
odd_postcode_verif

[9718]

In [58]:
df_dedup.loc[9718,"postcode_updated"]

'467l0'

In [59]:
df_dedup.loc[9718,"postcode_updated"]=0

In [60]:
#We assign the rest the zero value

for i in index_null_postcode:
    df_dedup.loc[i,"postcode_updated"]=0
for i in index_null_state:
    df_dedup.loc[i,"state_updated"]=0

In [61]:
df_dedup["postcode_updated"]=df_dedup["postcode_updated"].astype(int)
df_dedup.dtypes

patient_id            int64
given_name           object
surname              object
street_number       float64
address_1            object
suburb               object
postcode             object
state                object
date_of_birth       float64
age                 float64
phone_number         object
address_2            object
postcode_updated      int64
state_updated        object
suburb_updated       object
postcode_str         object
dtype: object

In [62]:
#We create two dictionnaries one for the mapping from postcode to states, the other one from states to postcode
postal_codes = dict([(i,"nt") for i in range(800,900)]+
                    [(i,"nsw") for i in range(1000,1999)]+
                    [(i,"nsw") for i in range(2000,2600)]+
                    [(i,"nsw") for i in range(2619,2900)]+
                    [(i,"act") for i in range(2600,2619)]+
                    [(i,"act") for i in range(2900,2921)]+
                    [(i,"nsw") for i in range(2921,3000)]+
                    [(i,"vic") for i in range(3000,4000)]+
                    [(i,"qld") for i in range(4000,5000)]+
                    [(i,"sa") for i in range(5000,5800)]+
                    [(i,"wa") for i in range(6000,6800)]+
                    [(i,"tas") for i in range(7000,7800)]
                   )
state_codes ={"nt":800,"nsw":1000,"act":2600,"vic":3000,"qld":4000,"sa":5000,"wa":6000,"tas":7000}
#For the state to postcode we assign the first value of the postcode range

In [63]:
#First case we update the state based on the non-null postcode
idx_not_null_postcode = [i for i in range(df_dedup.shape[0]) if df_dedup.loc[i,"postcode_updated"]!=0]

In [64]:
#Mapping the states to the dictionnary postal_codes
for i in idx_not_null_postcode:
    df_dedup.loc[i,"state_updated"]=postal_codes[df_dedup.loc[i,"postcode_updated"]]


In [65]:
#Verify
df_dedup.iloc[:5,5:]

,suburb,postcode,state,date_of_birth,age,phone_number,address_2,postcode_updated,state_updated,suburb_updated,postcode_str
0,ellenbrook,2527,wa,19710708.0,32.0,08 86018809,westella,2527,nsw,ellenbrook,2527
1,east preston,2074,nsw,19120921.0,34.0,02 97793152,foxdown,2074,nsw,east preston,2074
2,prospect,2305,nsw,19810905.0,22.0,02 20403934,None,2305,nsw,prospect,2305
3,henty,3620,wa,19840809.0,30.0,02 62832318,jodane,3620,vic,henty,3620
4,campbell town,4051,nsw,19340430.0,31.0,03 69359594,None,4051,qld,campbell town,4051


In [66]:
#Second case we update the null postcode based on the non-null state
idx_state_not_null=[]
for i in range(df_dedup.shape[0]):
    if df_dedup.loc[i,"state_updated"]!=0 and i not in idx_not_null_postcode:
        idx_state_not_null.append(i)

In [67]:
for i in idx_state_not_null:

    df_dedup.loc[i,"postcode_updated"]=state_codes[df_dedup.loc[i,"state_updated"]]

In [68]:
#Verify
df_dedup.iloc[95:100,5:]

,suburb,postcode,state,date_of_birth,age,phone_number,address_2,postcode_updated,state_updated,suburb_updated,postcode_str
95,coombabah,None,vic,19351020.0,26.0,03 90265874,hazel hill,3000,vic,coombabah,None
96,palmerston,4069,wa,19451003.0,NaN,04 65877742,None,4069,qld,palmerston,4069
97,harris park,3658,vic,19330804.0,21.0,02 04698047,ainslie nursing home,3658,vic,harris park,3658
98,burringbar,7330,wa,NaN,36.0,03 94670748,moorilla,7330,tas,burringbar,7330
99,labrador,2782,None,19710301.0,30.0,04 83500862,tongul,2782,nsw,labrador,2782


In [69]:
#Final case : the state and postcode of the entry is a null value
state_postcode_null = []
for i in range(df_dedup.shape[0]):
    if df_dedup.loc[i,"state_updated"]==0 and df_dedup.loc[i,"postcode_updated"]==0:
        state_postcode_null.append(i)

In [70]:
print("We have {} rows with no postcode and state".format(len(state_postcode_null)))

We have 17 rows with no postcode and state


In [71]:
#Like before we look at the suburbs
sb_list = [df_dedup.loc[i,"suburb"] for i in state_postcode_null]
set(sb_list)

{'alexandra bridge',
 'casino',
 'dapto',
 'downer',
 'elwood',
 'greenacre',
 'kings cross',
 'lake illawarra',
 'lower templestowe',
 'morven',
 'ngunnawal',
 'northmead',
 'salisbury',
 'wagaman',
 'winchelsea south',
 'winston hills',
 'yarrawonga'}

In [72]:
#The length of the set corresponds to the length of the suburbs list, all suburbs are unique.
#Because the list of the length is short we make a dictionnary
suburb_to_postcode = {"downer":2602,"wagaman":800,"kings cross":1340,"lake illawarra":2528,
                          "elwood":3184,
                          "greenacre":2190,
                          "salisbury":5108,
                          "northmead":2152,
                          "yarrawonga":3730,
                          "lower templestowe":3107,
                          "winston hills":2153,
                          "morven":4468,
                          "winchelsea south":3241,
                          "dapto":2530,
                          "alexandra bridge":6288,
                          "casino":2470,
                          "ngunnawal":2913}

In [73]:
#First we update the postcode from the suburb then the state from the postcode
for i in state_postcode_null:
    df_dedup.loc[i,"postcode_updated"]=suburb_to_postcode[df_dedup.loc[i,"suburb_updated"]]
    df_dedup.loc[i,"state_updated"]=postal_codes[df_dedup.loc[i,"postcode_updated"]]
 

In [74]:
df_dedup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
patient_id          20000 non-null int64
given_name          20000 non-null object
surname             20000 non-null object
street_number       20000 non-null float64
address_1           20000 non-null object
suburb              19788 non-null object
postcode            19801 non-null object
state               18010 non-null object
date_of_birth       17989 non-null float64
age                 16003 non-null float64
phone_number        20000 non-null object
address_2           20000 non-null object
postcode_updated    20000 non-null int64
state_updated       20000 non-null object
suburb_updated      19788 non-null object
postcode_str        20000 non-null object
dtypes: float64(3), int64(2), object(11)
memory usage: 2.4+ MB


In [75]:
#Finally we fill the null suburb with the string "None"
index_null_suburb = df_dedup[df_dedup['suburb'].isnull()].index.tolist()
for i in index_null_suburb:
    df_dedup.loc[i,"suburb_updated"]="None"

In [76]:
df_dedup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
patient_id          20000 non-null int64
given_name          20000 non-null object
surname             20000 non-null object
street_number       20000 non-null float64
address_1           20000 non-null object
suburb              19788 non-null object
postcode            19801 non-null object
state               18010 non-null object
date_of_birth       17989 non-null float64
age                 16003 non-null float64
phone_number        20000 non-null object
address_2           20000 non-null object
postcode_updated    20000 non-null int64
state_updated       20000 non-null object
suburb_updated      20000 non-null object
postcode_str        20000 non-null object
dtypes: float64(3), int64(2), object(11)
memory usage: 2.4+ MB


On peut maintenant passer à la correction des ages et date de naissances

### Age et Date de naissance

In [77]:
df_dedup['dob_updated']= df_dedup["date_of_birth"]
df_dedup['age_updated']= df_dedup["age"]

In [78]:
index_null_dob = df_dedup[df_dedup['dob_updated'].isnull()].index.tolist()

In [79]:
index_null_age = df_dedup[df_dedup['age_updated'].isnull()].index.tolist()

In [80]:
df_dedup["age_updated"]=df_dedup["age_updated"].fillna(0) #valeurs null
df_dedup["dob_updated"]=df_dedup["dob_updated"].fillna(0) #valeurs 

In [81]:
df_dedup['dob_updated']=df_dedup['dob_updated'].apply(lambda n: n if n%1 else int(n)) #remove trailling 0
df_dedup['age_updated']=df_dedup['age_updated'].apply(lambda n: n if n%1 else int(n)) #remove trailling 0

In [82]:
df_dedup["dob_updated"] = [(str(i)) for i in df_dedup["dob_updated"]]
df_dedup["dob_updated"] = [int(i[:4]) for i in df_dedup["dob_updated"]]

In [83]:
odd_year=df_dedup[(df_dedup["dob_updated"]<1900) & (df_dedup["dob_updated"]>0) ].index.tolist()

In [84]:
df_dedup.iloc[8882] 

patient_id                          258272
given_name                            jack
surname                             jessup
street_number                          509
address_1                      stone place
suburb                              batlow
postcode                              None
state                                  qld
date_of_birth                  1.29004e+07
age                                     34
phone_number                   08 77804659
address_2           mayflower retrmnt vlge
postcode_updated                      4000
state_updated                          qld
suburb_updated                      batlow
postcode_str                          None
dob_updated                           1290
age_updated                             34
Name: 8882, dtype: object

In [85]:
df_dedup.loc[8882,"dob_updated"]=2020-df_dedup.loc[8882,"age_updated"]

In [86]:
dob_zero = [i for i in range(df_dedup.shape[0]) if df_dedup.loc[i,"dob_updated"]==0 ]
age_zero= [i for i in  range(df_dedup.shape[0] )if df_dedup.loc[i,"age_updated"] ==0 ]
age_dob_zero =[i for i in dob_zero if i  in age_zero]
for i in age_dob_zero:
    df_dedup.loc[i,"dob_updated"]=1950
    df_dedup.loc[i,"age_updated"]=70

In [87]:
dob_zero_not_age =[i for i in dob_zero if i not in age_zero]
for i in dob_zero_not_age:
    df_dedup.loc[i,"dob_updated"]=2020-df_dedup.loc[i,"age_updated"]

In [88]:
dob_zero_verif = [i for i in range(df_dedup.shape[0]) if df_dedup.loc[i,"dob_updated"]==0 ]
age_zero_verif= [i for i in  range(df_dedup.shape[0] )if df_dedup.loc[i,"age_updated"] ==0 ]

In [89]:
df_dedup["age_final"]=df_dedup["age_updated"]

In [90]:
for i in range(df_dedup.shape[0]):
    df_dedup.loc[i,"age_final"]=2020-df_dedup.loc[i,"dob_updated"]

In [91]:
#Verify
df_dedup.iloc[1]

patient_id                771155
given_name                joshua
surname                   elrick
street_number                 23
address_1           andrea place
suburb              east preston
postcode                    2074
state                        nsw
date_of_birth        1.91209e+07
age                           34
phone_number         02 97793152
address_2                foxdown
postcode_updated            2074
state_updated                nsw
suburb_updated      east preston
postcode_str                2074
dob_updated                 1912
age_updated                   34
age_final                    108
Name: 1, dtype: object

In [92]:
cols_to_drop = ["suburb","postcode","state","date_of_birth","age","postcode_str","age_updated"]

In [93]:
df_dedup_clean = df_dedup.copy()

In [94]:
df_dedup_clean = df_dedup_clean.drop(["suburb","postcode","state","date_of_birth","age","postcode_str","age_updated"], axis = 1) 

In [95]:
df_dedup_clean

,patient_id,given_name,surname,street_number,address_1,phone_number,address_2,postcode_updated,state_updated,suburb_updated,dob_updated,age_final
0,221958,matisse,clarke,13.0,rene street,08 86018809,westella,2527,nsw,ellenbrook,1971,49
1,771155,joshua,elrick,23.0,andrea place,02 97793152,foxdown,2074,nsw,east preston,1912,108
2,231932,alice,conboy,35.0,mountain circuit,02 20403934,None,2305,nsw,prospect,1981,39
3,465838,sienna,craswell,39.0,cumberlegeicrescent,02 62832318,jodane,3620,vic,henty,1984,36
4,359178,joshua,bastiaans,144.0,lowrie street,03 69359594,None,4051,qld,campbell town,1934,86
5,744167,ky,laing,448.0,nyawi place,03 59872070,None,3556,vic,barmera,1905,115
6,210268,matthew,laing,11.0,barnes place,02 86925029,None,2160,nsw,laurieton,1906,114
7,832180,jack,renfrey,27.0,osmand street,03 15575583,dhurringill,2170,nsw,maribyrnong,1961,59
8,154886,adele,ryan,76.0,house circuit,07 37444521,None,2200,nsw,new farm,1943,77
9,237337,breeanne,wynne,12.0,cowper street,08 24888117,None,2062,nsw,bonnet bay,1903,117


In [96]:
df_dedup_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
patient_id          20000 non-null int64
given_name          20000 non-null object
surname             20000 non-null object
street_number       20000 non-null float64
address_1           20000 non-null object
phone_number        20000 non-null object
address_2           20000 non-null object
postcode_updated    20000 non-null int64
state_updated       20000 non-null object
suburb_updated      20000 non-null object
dob_updated         20000 non-null int64
age_final           20000 non-null int64
dtypes: float64(1), int64(4), object(7)
memory usage: 1.8+ MB


On fait le choix de garder ces colonnes pour nous permettre d'avoir un choix multiples en terme de critères de sélection pour la détection des doublons dans un dataframe.

In [97]:
#Export dataframe to csv
#df_dedup_clean.to_csv('df_dedup_clean.csv',index=False)

## 1. Nettoyage du dataframe : ***df_pcr***

In [98]:
df_pcr.describe()

,patient_id
count,8800.000000
mean,554175.918977
std,259491.959500
min,100037.000000
25%,331553.000000
50%,556969.000000
75%,778063.750000
max,999993.000000


In [99]:
print("On a donc {} ID de patients unique et {} doublons d'ID de patients".format(len(df_pcr["patient_id"].unique()),len(df_pcr)-len(df_pcr["patient_id"].unique())))


On a donc 8765 ID de patients unique et 35 doublons d'ID de patients


In [100]:
df_pcr[df_pcr.duplicated(keep=False)].sort_values(by="patient_id")

,patient_id,pcr
169,159500,N
2934,159500,N
1473,227192,N
8249,227192,N
3251,328071,Positive
4741,328071,Positive
4939,411955,N
3569,411955,N
1265,465871,N
7254,465871,N


Les 35 doublons d'ID de patients ont les même résultats PCR, on peut donc les supprimer. 

In [101]:
df_pcr = df_pcr.drop_duplicates(keep=False)
df_pcr.sort_values(by="patient_id")

,patient_id,pcr
1900,100037,N
3919,100215,Negative
1733,100294,N
1672,100363,P
1880,100404,N
6336,100727,Negative
3699,100882,Positive
4592,100917,Negative
4703,101051,Negative
6005,101082,Positive


In [102]:
df_pcr.describe(include=['object'])

,pcr
count,8772
unique,4
top,N
freq,3468


Pour le résultat du PCR on a donc 4 valeurs possibles : Negative, Postive, N ou P. Ces 4 valeurs sont similaires on peut donc les uniformiser pour avoir uniquement Positive et Negative, la colonne PCR renvoyant une valeur soit True(Positive,1) soit False (Negative,0)

In [103]:
bool_pcr = {"Negative":False, "N":False, "Positive":True, "P":True}

In [104]:
df_pcr_clean = df_pcr.copy()

In [105]:
df_pcr_clean =df_pcr_clean.replace({"pcr":bool_pcr})

In [106]:
df_pcr_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8772 entries, 0 to 8799
Data columns (total 2 columns):
patient_id    8772 non-null int64
pcr           8772 non-null bool
dtypes: bool(1), int64(1)
memory usage: 145.6 KB


In [107]:
df_pcr_clean.head(5)

,patient_id,pcr
0,653091,False
1,347667,False
2,708481,True
3,148825,False
4,150081,False


In [108]:
#Export dataframe to csv
#df_pcr_clean.to_csv('df_pcr_clean_final.csv',index=False)

In [109]:
#Merge two dataframe based on the patient ID key
df_patient_pcr = pd.merge(df_pcr_clean,df_dedup_clean,on="patient_id",how="inner")
#We want only the patient who were tested, so we only keep rows where the patient id value exists in both pcr and
#patient dataframe

In [110]:
df_patient_pcr

,patient_id,pcr,given_name,surname,street_number,address_1,phone_number,address_2,postcode_updated,state_updated,suburb_updated,dob_updated,age_final
0,653091,False,daniel,campbell,58.0,sid barnes crescent,08 38772117,None,4514,qld,north ward,1973,47
1,347667,False,sebastian,mchenry,12.0,mundawari circuit,08 61083524,ocean star villas,4551,qld,swan view,1990,30
2,708481,True,sarah,rellos,20.0,torres street,02 73197286,None,3199,vic,bribie island,1943,77
3,148825,False,chloe,brammy,238.0,fitchett street,02 48826642,None,3280,vic,carnegie,1954,66
4,150081,False,charlie,None,6.0,clark close,03 24096098,talawa,2602,act,south melbourne,1975,45
5,876796,False,alexander,dixon,11.0,None,04 21654709,woodberry vlge,2142,nsw,torquay,1918,102
6,169224,False,kyle,eyles,79.0,longworth place,0,hopeview,2527,nsw,mentone,1938,82
7,755915,True,jamie,krollig,72.0,kingsford smith drive,08 03345937,None,4224,qld,toowong,1961,59
8,374486,False,rachael,wilkins,17.0,kelsall place,08 50445222,cascina,3194,vic,bayswater,2007,13
9,634003,False,lauren,braunack,2.0,lumeah street,08 76845890,inglewood,3315,vic,hamilton,1930,90


In [111]:
#Export dataframe to csv
#df_pcr_clean.to_csv('df_patient_pcr_clean.csv',index=False)